# ReAct

In [1]:
from openai import OpenAI
from typing import List, Dict, Optional, Union

def ask(messages: List[Dict],is_json:bool = False, stop: Optional[Union[str, List]] = None):
        client = OpenAI(api_key='YOUR OPENAI KEY')

        if is_json:
                response = client.chat.completions.create(
                        model = 'gpt-4o-mini-2024-07-18',
                        temperature = 0.7,
                        messages = messages,
                        response_format={ "type": "json_object" },
                        stop = stop
                )
        else:
                response = client.chat.completions.create(
                        model = 'gpt-4o-mini-2024-07-18',
                        temperature = 0.7,
                        messages = messages,
                        stop = stop
                )

        return response

In [2]:
## Web Search
from search import internet_search, process_content

class WebSearch:
    def __init__(self, name:str='web_search', threhold:int=8000):
        self.system_prompt = """
You are a Insight Researcher.

1. To find detail informtion for the user query
and summary the content into one sentence as simple as possible
2. If the user's question is about specific numerical values, 
only return the numerical results without any additional explanation.
"""
        self.name = name
        self.description = "the tool use for web search"
        self.threhold = threhold

    def __call__(self, query:str):
        results = internet_search(query)
        all_text = ""
        windows_size = 0
        for item in results:
            if windows_size >= self.threhold:
                break
            page_content = process_content(item['href'])
            for page in page_content:
                if windows_size + len(page) > self.threhold:
                    remaining_space = self.threhold - windows_size
                    all_text += page[:remaining_space].strip() + "\n\n"
                    windows_size = self.threhold
                    break
                else:
                    windows_size += len(page)
                    all_text += page + "\n\n"
            if windows_size >= self.threhold:
                break

        msg = [{"role":"system","content":self.system_prompt},
               {"role":"user", "content": f"The query is {query}, The search results are {all_text}"}]
        
        answer = ask(messages=msg)

        return answer.choices[0].message.content

In [3]:
# system prompt
react_prompt = """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
"""

In [4]:
query = "2024 UEFA European Championship and 2024 Copa América champion"

In [5]:
search = WebSearch()
available_tools = {search.name: search}

In [6]:
# constructure prompt
user_prompt = react_prompt.format(tools=search.description, 
                                    tool_names=search.name,
                                    input=query,
                                    agent_scratchpad='')
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion



In [7]:
# 1st ask llm
messages = [
    {"role":"user", "content":user_prompt},
]

result1 = ask(messages=messages,
              stop=["Observation", " Observation"])
intermediate_step1 = result1.choices[0].message.content
print(intermediate_step1)

Thought: I need to find out the champions of the 2024 UEFA European Championship and the 2024 Copa América, as these events are in the future and their outcomes are not yet known. I will perform a web search to find the latest information regarding these tournaments. 
Action: web_search
Action Input: "2024 UEFA European Championship champion" 



In [8]:
import re
# parse the 1st result
regex = (
            r"Action\s*\d*\s*:[\s]*(.*?)[\s]*Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        )
action_match = re.search(regex, intermediate_step1, re.DOTALL)
action = action_match.group(1).strip()
action_input = action_match.group(2)
tool_input = action_input.strip(" ").strip('\n').strip('"')
print(action)
print(tool_input)

web_search
2024 UEFA European Championship champion" 


In [9]:
# call tool function
tool_function1 = available_tools[action]
print(tool_input)
tool_result1 = tool_function1(query=str(tool_input))
print(tool_result1)

2024 UEFA European Championship champion" 
Spain won the 2024 UEFA European Championship, defeating England 2-1 in the final.


In [10]:
user_prompt += intermediate_step1
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: I need to find out the champions of the 2024 UEFA European Championship and the 2024 Copa América, as these events are in the future and their outcomes are not yet known. I will perform a web search to find the latest information regarding these tournaments. 
Action: web_search
Action Input: "2024 UEFA European Championship cham

In [11]:
user_prompt += f"Observation: {tool_result1}\nThought:"
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: I need to find out the champions of the 2024 UEFA European Championship and the 2024 Copa América, as these events are in the future and their outcomes are not yet known. I will perform a web search to find the latest information regarding these tournaments. 
Action: web_search
Action Input: "2024 UEFA European Championship cham

In [12]:
# 2ed ask llm
messages = [
    {"role":"user", "content":user_prompt},
]

result2 = ask(messages=messages,
              stop=["Observation", " Observation"])
intermediate_step2 = result2.choices[0].message.content
print(intermediate_step2)

I have found the champion of the 2024 UEFA European Championship; Spain won the tournament. Now I need to find out the champion of the 2024 Copa América. 
Action: web_search
Action Input: "2024 Copa América champion" 



In [13]:
user_prompt += intermediate_step2
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: I need to find out the champions of the 2024 UEFA European Championship and the 2024 Copa América, as these events are in the future and their outcomes are not yet known. I will perform a web search to find the latest information regarding these tournaments. 
Action: web_search
Action Input: "2024 UEFA European Championship cham

In [14]:
import re
# parse the 1st result
regex = (
            r"Action\s*\d*\s*:[\s]*(.*?)[\s]*Action\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        )
action_match = re.search(regex, intermediate_step2, re.DOTALL)
action = action_match.group(1).strip()
action_input = action_match.group(2)
tool_input = action_input.strip(" ").strip('\n').strip('"')
print(action)
print(tool_input)
# call tool function
tool_function2 = available_tools[action]
print(tool_function2.name)
print(tool_input)
tool_result2 = search(query=str(tool_input))
print(tool_result2)

web_search
2024 Copa América champion" 
web_search
2024 Copa América champion" 
Argentina won the 2024 Copa América, defeating Colombia 1-0 with a late goal in the 112th minute.


In [15]:
user_prompt += f"Observation: {tool_result2}\nThought:"
print(user_prompt)


Answer the following questions as best you can. You have access to the following tools:

the tool use for web search

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [web_search]
Action Input: the input to the action, return format like `action_input`
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 2024 UEFA European Championship and 2024 Copa América champion
Thought: I need to find out the champions of the 2024 UEFA European Championship and the 2024 Copa América, as these events are in the future and their outcomes are not yet known. I will perform a web search to find the latest information regarding these tournaments. 
Action: web_search
Action Input: "2024 UEFA European Championship cham

In [16]:
# 3rd call LLM
messages = [
    {"role":"user", "content":user_prompt},
]

result3 = ask(messages=messages,
              stop=["Observation", " Observation"])
intermediate_step3 = result3.choices[0].message.content
print(intermediate_step3)

I have found the champion of the 2024 Copa América; Argentina won the tournament. 

Final Answer: Spain won the 2024 UEFA European Championship, and Argentina won the 2024 Copa América.
